In [1]:
from openai import OpenAI

openai_client = OpenAI()

In [5]:
import json

from gitsource import GithubRepositoryDataReader, chunk_documents
from minsearch import AppendableIndex


reader = GithubRepositoryDataReader(
    repo_owner="evidentlyai",
    repo_name="docs",
    allowed_extensions={"md", "mdx"},
)
files = reader.read()

parsed_docs = [doc.parse() for doc in files]
chunked_docs = chunk_documents(parsed_docs, size=3000, step=1500)

index = AppendableIndex(
    text_fields=["title", "description", "content"],
    keyword_fields=["filename"]
)
index.fit(chunked_docs)

In [6]:
def search(query):
    results = index.search(
        query=query,
        num_results=5
    )
    return results

search_tool = {
    "type": "function",
    "name": "search",
    "description": "Search the documentation database for relevant results based on a query string.",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "The search query to look up in the index"
            }
        },
        "required": [
            "query"
        ]
    }
}

In [39]:
def make_call(tool_call):
    arguments = json.loads(tool_call.arguments)
    name = tool_call.name

    if name == 'search':
        result = search(**arguments)
    # elif name == 'add_entry':
    #     result = add_entry(**arguments)
    else: 
        result = 'not found tool "{name}"'
    
    return {
        "type": "function_call_output",
        "call_id": tool_call.call_id,
        "output": json.dumps(result),
    }

In [32]:
instructions = """
You're a documentation assistant. 

Answer the user question using the documentation knowledge base

IMPORTANT: When you explore the knowledge base, make at least 3 different
searchers to make sure you explore the topic well.

Use only facts from the knowledge base when answering.
If you cannot find the answer, inform the user.

Our knowledge base is entirely about Evidently, so you don't need to 
include the word 'evidently' in search results
"""

In [33]:
question = "How do I create a dahsbord in Evidently?"

In [34]:
message_history = [
    {"role": "system", "content": instructions},
    {"role": "user", "content": question}
]

In [35]:

response = openai_client.responses.create(
    model='gpt-4o-mini',
    input=message_history,
    tools=[search_tool],
)


In [36]:
message_history.extend(response.output)

In [37]:
message_history

[{'role': 'system',
  'content': "\nYou're a documentation assistant. \n\nAnswer the user question using the documentation knowledge base\n\nIMPORTANT: When you explore the knowledge base, make at least 3 different\nsearchers to make sure you explore the topic well.\n\nUse only facts from the knowledge base when answering.\nIf you cannot find the answer, inform the user.\n\nOur knowledge base is entirely about Evidently, so you don't need to \ninclude the word 'evidently' in search results\n"},
 {'role': 'user', 'content': 'How do I create a dahsbord in Evidently?'},
 ResponseFunctionToolCall(arguments='{"query":"create dashboard"}', call_id='call_RbeSGbIIlJB5DdCAI1WT6qUl', name='search', type='function_call', id='fc_002756e15a4d35e6006989c168ad408195824819f71ae6756c', status='completed'),
 ResponseFunctionToolCall(arguments='{"query":"dashboard creation"}', call_id='call_Z50EohvLyLovH3wyynqR1Nu0', name='search', type='function_call', id='fc_002756e15a4d35e6006989c168ad5081959998096820

In [42]:
for message in response.output:
    if message.type == 'function_call':
        print(f'executing {message.name}({message.arguments})...')
        tool_call_output = make_call(message)
        message_history.append(tool_call_output)

executing search({"query":"create dashboard"})...
executing search({"query":"dashboard creation"})...
executing search({"query":"dashboard features"})...


In [44]:
len(message_history)

8

In [45]:
response = openai_client.responses.create(
    model='gpt-4o-mini',
    input=message_history,
    tools=[search_tool],
)

response.usage.input_tokens

12235

In [48]:
response.output[0].type

'message'

In [46]:
print(response.output_text)

To create a dashboard in Evidently, follow these steps:

### 1. **Create a Project**
Before you can create a dashboard, ensure you've created a project and added reports with evaluation results.

### 2. **Add Tabs**
You can organize your dashboard by adding multiple tabs:
- **Enter "Edit" mode** on the dashboard (top right corner).
- Click the **plus sign** with "add Tab" on the left.
- **Choose an option** to create a custom tab or select a pre-built tab.

### 3. **Add Panels**
Panels visualize evaluation results over time. Here's how to add them:
- While in **Edit mode**, click the **"Add Panel"** button.
- **Configure the panel** by selecting the metrics you want to visualize:
  - **Select Metrics** relevant to your reports.
  - Optionally, filter by tags if necessary.

### 4. **Panel Types**
You can create various types of panels, including:
- **Text panels** for titles.
- **Counters** to show metrics such as totals or averages.
- **Pie charts**, **line plots**, and **bar plots** (

In [59]:
instructions = """
You're a documentation assistant. 

Answer the user question using the documentation knowledge base

Make 3 iterations:

1) in the first iteration, perform one search
2) in the second interation, analyze the results from the previous search
   and perform 2 more searches
3) synthesise the results into the output

IMPORTANT: at each step, give an explanation of why you want to perform 
search for this particular search query. It should be 2-3 sentences explaining
the logic of your decision.

Use only facts from the knowledge base when answering.
If you cannot find the answer, inform the user.

Our knowledge base is entirely about Evidently, so you don't need to 
include the word 'evidently' in search results
"""

In [60]:
question = "How do I create a dahsbord in Evidently?"

In [61]:
message_history = [
    {"role": "system", "content": instructions},
    {"role": "user", "content": question}
]

iteration_number = 1

while True:
    response = openai_client.responses.create(
        model='gpt-4o-mini',
        input=message_history,
        tools=[search_tool],
    )

    print(f'iteraration number {iteration_number}...') 
    message_history.extend(response.output)

    has_function_calls = False

    for message in response.output:
        if message.type == 'function_call':
            print(f'executing {message.name}({message.arguments})...')
            tool_call_output = make_call(message)
            message_history.append(tool_call_output)
            has_function_calls = True

        if message.type == 'message':
            text = message.content[0].text
            print('ASSISTANT:', text)

    iteration_number = iteration_number + 1
    print()
    
    if not has_function_calls:
        break

iteraration number 1...
ASSISTANT: I will first search for information regarding dashboard creation in order to provide you with the basic steps or guidelines. Dashboards are essential for data visualization and reporting, so it's important to find accurate instructions on how to set one up.

Let's begin with the initial search for "create dashboard."
executing search({"query":"create dashboard"})...

iteraration number 2...
ASSISTANT: In the initial search, I found detailed information about creating dashboards, including adding tabs and panels, as well as the necessary requirements to populate them with data. This information lays the groundwork for understanding how to create a functional dashboard.

### Steps to Create a Dashboard

1. **Create a Project**: You must first connect to the platform and create a project if you haven't already done so.
   
2. **Enter Edit Mode**: To start adding elements to your dashboard, enter "Edit" mode.

3. **Adding Tabs**:
   - Click the plus sign 

In [65]:
def add_entry(filename, title, description, content):
    entry = {
        'start': 0,
        'content': content,
        'title': title,
        'description': description,
        'filename': filename,
    }
    index.append(entry)
    return "OK"

add_entry_tool = {
    "type": "function",
    "name": "add_entry",
    "description": "Add a new documentation entry to the index.",
    "parameters": {
        "type": "object",
        "properties": {
            "filename": {
                "type": "string",
                "description": "The source filename associated with the entry"
            },
            "title": {
                "type": "string",
                "description": "The title of the documentation entry"
            },
            "description": {
                "type": "string",
                "description": "A short description summarizing the entry"
            },
            "content": {
                "type": "string",
                "description": "The full content of the documentation entry"
            }
        },
        "required": [
            "filename",
            "title",
            "description",
            "content"
        ]
    }
}


In [66]:
def make_call(tool_call):
    arguments = json.loads(tool_call.arguments)
    name = tool_call.name

    if name == 'search':
        result = search(**arguments)
    elif name == 'add_entry':
        result = add_entry(**arguments)
    else: 
        result = 'not found tool "{name}"'
    
    return {
        "type": "function_call_output",
        "call_id": tool_call.call_id,
        "output": json.dumps(result),
    }

In [67]:
message_history = [
    {"role": "system", "content": instructions},
    {"role": "user", "content": question}
]
tools = [search_tool, add_entry_tool]

iteration_number = 1

while True:
    response = openai_client.responses.create(
        model='gpt-4o-mini',
        input=message_history,
        tools=tools,
    )

    print(f'iteraration number {iteration_number}...') 
    message_history.extend(response.output)

    has_function_calls = False

    for message in response.output:
        if message.type == 'function_call':
            print(f'executing {message.name}({message.arguments})...')
            tool_call_output = make_call(message)
            message_history.append(tool_call_output)
            has_function_calls = True

        if message.type == 'message':
            text = message.content[0].text
            print('ASSISTANT:', text)

    iteration_number = iteration_number + 1
    print()
    
    if not has_function_calls:
        break

iteraration number 1...
ASSISTANT: To provide a comprehensive answer on creating a dashboard, I'll first search for general guidance on dashboard creation within the documentation. This step is essential to understand the foundational process and requirements involved in dashboard creation.

Let's conduct the first search.
executing search({"query":"create dashboard"})...

iteraration number 2...
ASSISTANT: The initial search results provide detailed instructions on creating dashboards and adding panels in the user interface as well as using the Python API. Here are the steps involved based on the documentation:

1. **Add Tabs**: 
   - Enter "Edit" mode on the Dashboard.
   - Click the plus sign to add a new Tab. You can also choose a "pre-built Tab" template.

2. **Add Panels**:
   - Enter "Edit" mode on the Dashboard.
   - Click the "Add Panel" button.
   - Configure your panel by selecting metrics, adjusting plot types, and setting aggregation methods. 

This gives an overview of th

In [68]:
message_history.append(
    {"role": "user", "content": "add this content to our database"}
)

In [69]:
while True:
    response = openai_client.responses.create(
        model='gpt-4o-mini',
        input=message_history,
        tools=tools,
    )

    print(f'iteraration number {iteration_number}...') 
    message_history.extend(response.output)

    has_function_calls = False

    for message in response.output:
        if message.type == 'function_call':
            print(f'executing {message.name}({message.arguments})...')
            tool_call_output = make_call(message)
            message_history.append(tool_call_output)
            has_function_calls = True

        if message.type == 'message':
            text = message.content[0].text
            print('ASSISTANT:', text)

    iteration_number = iteration_number + 1
    print()
    
    if not has_function_calls:
        break

iteraration number 4...
executing add_entry({"filename":"docs/platform/dashboard_creation.mdx","title":"Creating a Dashboard","description":"Step-by-step guide to creating a dashboard, including adding tabs and panels.","content":"### Steps to Create a Dashboard\n\n1. **Create Your Dashboard**:\n   - Navigate to the dashboard creation area and enter \"Edit\" mode.\n\n2. **Add Tabs**:\n   - Click the \"+ Add Tab\" button.\n   - Choose to create a custom tab or start with a pre-built tab. Name your tab appropriately.\n\n3. **Add Panels**:\n   - Click on the \"Add Panel\" button.\n   - Select the type of panel you wish to create (text, counter, pie chart, line, or bar).\n   - Configure the panel by setting:\n     - **Title**: Provide a meaningful title.\n     - **Size**: Choose between \"full\" or \"half\".\n     - **Values**: Specify metrics using `PanelMetric`.\n     - **Plot Parameters**: Adjust visual settings (e.g., chart type).\n\n4. **Save and Organize**:  \n   - Review your panel 

In [70]:
index.docs[-1]

{'start': 0,
 'content': '### Steps to Create a Dashboard\n\n1. **Create Your Dashboard**:\n   - Navigate to the dashboard creation area and enter "Edit" mode.\n\n2. **Add Tabs**:\n   - Click the "+ Add Tab" button.\n   - Choose to create a custom tab or start with a pre-built tab. Name your tab appropriately.\n\n3. **Add Panels**:\n   - Click on the "Add Panel" button.\n   - Select the type of panel you wish to create (text, counter, pie chart, line, or bar).\n   - Configure the panel by setting:\n     - **Title**: Provide a meaningful title.\n     - **Size**: Choose between "full" or "half".\n     - **Values**: Specify metrics using `PanelMetric`.\n     - **Plot Parameters**: Adjust visual settings (e.g., chart type).\n\n4. **Save and Organize**:  \n   - Review your panel configurations in the preview and click "Save".\n   - Ensure the correct tab is selected for the new panel.\n\n5. **Adjust as Needed**:  \n   - You can later edit or delete panels and tabs using the "Edit" mode.',
 